# `pyGIMLi(emg3d)`

Requires
- `simpeg>=1.5.2`
- `emg3d@inv-pygimli` (`pip install git+https://github.com/emsig/emg3d@inv-pygimli`

Quite simple plus all possible inv-crimes (same mesh, some solver, ...).

**Current Limitations**
- Only isotropic models supported
- Only 'Conductivity' models supported
- Without el. perm. and magn. perm.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import emg3d
# import pygimli as pg
import emg3d.inversion.pygimli as ipygimli

%run ../data/sampledata

In [ ]:
%matplotlib widget

## Load Simulation (incl. Survey, Models, Data)

In [ ]:
sim, model = load_data("../data/pyGIMLi-Original")
model_init = sim.model.copy()

# sim.tol_gradient = 1e-2  # could be reduced further to potentially speed-up more

## Define regions, instantiate Kernel and Inversion

In [ ]:
markers = np.zeros(sim.model.shape, dtype=int)
markers[:, :, sim.model.grid.cell_centers_z > 0] = 1

# Instantiate the emg3d-kernel for pyGIMLi
fop = ipygimli.Kernel(simulation=sim, markers=markers, pgthreads=1)

# Instantiate the inversion and set some inversion parameters
INV = ipygimli.Inversion(fop=fop)
INV.inv.setCGLSTolerance(10)  # is _absolute_, not _relative_
INV.inv.setMaxCGLSIter(30)

# Set the regions.
INV.fop.setRegionProperties(0, limits=(0.001, 2), startModel=1)
INV.fop.setRegionProperties(1, limits=(3, 4), startModel=3.33)

# INV.stopAtChi1 = False

In [ ]:
# DIFFERENT # fop.obs_resp_errors[fop.obs_resp_errors > 0.5] = 1e8
INV.run(
    # dataVals=fop.obs_resp_data,  # DIFFERENT # 
    # relativeError=fop.obs_resp_errors,  # DIFFERENT # 
    maxIter=20, # just to test
    lam=1,
    #verbose=True,
    #startModel=...,
    #isReference=True,
)

## Plots

In [ ]:
plot_models(sim, sim.model, model, zind=4, vmin=0.3, vmax=100)

In [ ]:
plot_responses(sim)

In [ ]:
emg3d.Report()